In [7]:
audio_file = "audio/5760-Nano-L2.mp3"

In [ ]:
import torch
from transformers import pipeline
import time

# Path to the audio file
audio_file = "audio/5760-Nano-L2.mp3"

# Check if Metal Performance Shaders (MPS) is available for GPU acceleration
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Metal GPU acceleration")
else:
    device = torch.device("cpu")
    print("Metal not available, using CPU")

# Set up the automatic speech recognition pipeline
pipe = pipeline("automatic-speech-recognition", 
                model="openai/whisper-large-v3",  # Using the large-v3 Whisper model
                torch_dtype=torch.float16,  # Using half-precision for efficiency
                device=device,  # Using the device determined above
                return_timestamps=True)  # Enable timestamp generation

# Start timing the transcription process
start_time = time.time()

# Perform the transcription
result = pipe(audio_file)

# Calculate the elapsed time
elapsed_time = time.time() - start_time

# Print the results
print(f"Transcription completed in {elapsed_time:.2f} seconds")
print("Transcription result:")
print(result["text"])

# Print device and model information for debugging
print(f"Device: {pipe.device}")
print(f"Model dtype: {pipe.model.dtype}")

# Function to process long audio files in chunks
def process_in_chunks(audio_file, chunk_length_s=30):
    from pydub import AudioSegment
    import math
    import os

    # Load the audio file
    audio = AudioSegment.from_mp3(audio_file)
    
    # Calculate the number of chunks
    chunk_length_ms = chunk_length_s * 1000
    chunks = math.ceil(len(audio) / chunk_length_ms)

    transcriptions = []

    for i in range(chunks):
        # Extract a chunk of audio
        start_time = i * chunk_length_ms
        end_time = (i + 1) * chunk_length_ms
        chunk = audio[start_time:end_time]
        
        # Save the chunk as a temporary WAV file
        chunk_file = f"temp_chunk_{i}.wav"
        chunk.export(chunk_file, format="wav")

        # Transcribe the chunk
        result = pipe(chunk_file, language="en")
        transcriptions.append(result["text"])

        # Clean up the temporary file
        os.remove(chunk_file)

    # Join all transcriptions into a single string
    return " ".join(transcriptions)

# Process the entire audio file in chunks
full_transcription = process_in_chunks(audio_file)
print("Full transcription:")
print(full_transcription)


Using Metal GPU acceleration


Device set to use mps
/opt/anaconda3/envs/transcribe311/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
